In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
df1 = pd.read_excel('2020_fb_pres_1000_topic.xlsx')
df2 = pd.read_excel('calculate_f1_1000ad.xlsx')
df3 = pd.read_excel('500_ads_final_cleaned.xlsx')
df4 = pd.read_excel('calculate_f1_500ad.xlsx')

In [9]:
df3.columns

Index(['_id', 'ad_creation_time', 'Unnamed: 2', 'ad_creative_link_caption',
       'ad_creative_link_description', 'ad_creative_link_title',
       'ad_delivery_start_time', 'ad_delivery_stop_time', 'ad_snapshot_url',
       'currency', 'demographic_distribution', 'funding_entity', 'id',
       'impressions', 'page_id', 'page_name', 'publisher_platforms',
       'region_distribution', 'spend', 'ad_creative_body', 'attack', 'image',
       'Gold Label'],
      dtype='object')

In [10]:
df4.columns

Index(['economic', 'environment', 'foreign policy', 'governance',
       'immigration', 'military', 'safety', 'social programs',
       'social and cultural', 'economic_pred', 'environment_pred',
       'foreign policy_pred', 'governance_pred', 'immigration_pred',
       'military_pred', 'safety_pred', 'social programs_pred',
       'social and cultural_pred'],
      dtype='object')

In [8]:
df1['label'] = df2['Gold Label']
df1 = df1[['text','label']]
df1.head()

,text,label
0,The Gulf of Oman incident must not be used as ...,foreign policy
1,"Thanks for having me, Girlboss! I love a good ...","economic,social and cultural"
2,For a problem as massive and urgent as global ...,environment
3,My friend Hope is very clear that women’s hist...,"economic,social and cultural"
4,"In just ten days, I'll take the stage for the ...",social programs


In [13]:
df3 = df3[['ad_creative_body','Gold Label']]
df3.rename(columns={'ad_creative_body':'text','Gold Label':'label'},inplace=True)
df3.head()

,text,label
0,"Last quarter, Bernie Sanders, Pete Buttigieg a...",none
1,Every one of the Democratic candidates remaini...,none
2,A young woman came up to me after my speech in...,social programs
3,"Como presidenta, Elizabeth proporcionarн_a el ...",Social Programs
4,Una cosa queda clara: ahorrar para una jubilac...,Social Programs


In [14]:
df = pd.concat([df1,df3])
df.shape

(1499, 2)

In [19]:
# import the stop words from mallet
stopwords = pd.read_csv(r'C:\Users\wangtao\Documents\Graviry_Spy Projects\original data\mallet.txt',sep=' ',header=None)
stopwords = stopwords[0].tolist()

In [22]:
def filtered (x):
    # regular expression for url
    #url = r'[https]{5}\\:\\/\\/([a-zA-Z]|[0-9])*(\\.([a-zA-Z]|[0-9])*)*(\\/([a-zA-Z]|[0-9])*)*\\s?'
    #url = r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))'''
    url = 'http[s]?://\S+'
    x = re.sub(url,'',x)
    x = re.sub("[^\w\s]",' ',x)
    x = re.sub("\s+",' ',x)
    
    ls=[w.lower() for w in x.split() if w.lower() not in stopwords]
    
    return ' '.join(ls)
    

In [15]:
# df.to_excel('1500_FB_ads_GoldLabel.xlsx')

In [25]:
df5['text'] = df5['text'].astype(str).apply(lambda x: filtered(x))

In [43]:
# df5 = pd.read_excel('1500_FB_ads_GoldLabel.xlsx')
df5 = pd.read_csv('1500_FB_ads_GoldLabel.csv')

In [44]:
df5.head()

,text,label
0,gulf oman incident pretext war iran war unmiti...,foreign policy
1,girlboss love good conversation women sideline...,"economic,social and cultural"
2,problem massive urgent global climate change s...,environment
3,friend hope clear women history month matters ...,"economic,social and cultural"
4,ten days ll stage debate 2 3 democratic voters...,social programs


In [45]:
topics=['economic', 'environment', 'foreign policy', 'governance', 'immigration', 'military', 'safety','social programs', 'social and cultural']

In [46]:
data0 = {x:np.zeros(1499,dtype=int) for x in topics}

table0= pd.DataFrame(data0)

df5 = pd.concat([df5,table0],axis=1)

df5.tail()

,text,label,economic,environment,foreign policy,governance,immigration,military,safety,social programs,social and cultural
1494,medicare ensure granite stater covered costs w...,"economic, social programs",0,0,0,0,0,0,0,0,0
1495,america that???? working thinner thinner slice...,no topic,0,0,0,0,0,0,0,0,0
1496,election lobbyists billionaires buy politician...,governance,0,0,0,0,0,0,0,0,0
1497,system money influence helped stage secrecy le...,"foreign policy, governance",0,0,0,0,0,0,0,0,0
1498,election lobbyists billionaires buy politician...,governance,0,0,0,0,0,0,0,0,0


In [47]:
for index, row in df5.iterrows():
    #print(index)
    #print(row)
    
    if len(row['label'].split(',')) ==1 and row['label']!= 'no topic':
        df5[row['label'].strip()][index]=1
        
    if len(row['label'].split(','))>1:
        for w in row['label'].split(','):
            #print(index)
            df5[w.strip()][index]=1
    
    else: 
        pass

C:\Users\wangtao\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\wangtao\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [48]:
df5.head()

,text,label,economic,environment,foreign policy,governance,immigration,military,safety,social programs,social and cultural
0,gulf oman incident pretext war iran war unmiti...,foreign policy,0,0,1,0,0,0,0,0,0
1,girlboss love good conversation women sideline...,"economic,social and cultural",1,0,0,0,0,0,0,0,1
2,problem massive urgent global climate change s...,environment,0,1,0,0,0,0,0,0,0
3,friend hope clear women history month matters ...,"economic,social and cultural",1,0,0,0,0,0,0,0,1
4,ten days ll stage debate 2 3 democratic voters...,social programs,0,0,0,0,0,0,0,1,0


In [49]:
# df5.to_csv('1500_FB_ads_GoldLabel.csv',index=False)